In [1]:
%cd /home/davidhenao695/Python/Backend/FastAPI/video_membership

/home/davidhenao695/Python/Backend/FastAPI/video_membership


/home/davidhenao695/Python/Backend/FastAPI/video_membership/venv/lib/python3.11/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
!pwd # path

/home/davidhenao695/Python/Backend/FastAPI/video_membership


In [3]:
from app.db.models import Video, Playlist
from sqlmodel import select
from app.db.database import async_session
from sqlmodel.ext.asyncio.session import AsyncSession
from pydantic import BaseModel, Field, field_validator, model_validator
from uuid import UUID
from typing import Optional

session: AsyncSession  =  async_session()

In [4]:
statement = select (Video)
video_db = await session.exec(statement)
video = video_db.first()
video_dict = video.__dict__

2024-07-03 12:11:24,389 INFO sqlalchemy.engine.Engine select pg_catalog.version()
2024-07-03 12:11:24,393 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-07-03 12:11:25,294 INFO sqlalchemy.engine.Engine select current_schema()
2024-07-03 12:11:25,295 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-07-03 12:11:26,822 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2024-07-03 12:11:26,824 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-07-03 12:11:27,492 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-07-03 12:11:27,510 INFO sqlalchemy.engine.Engine SELECT videos.db_id, videos.host_service, videos.host_id, videos.url, videos.title, videos.owner_id 
FROM videos
2024-07-03 12:11:27,512 INFO sqlalchemy.engine.Engine [generated in 0.00236s] ()


In [5]:
class VideoIndex(BaseModel):
    objectID : str = Field(alias='host_id')
    title : Optional[str]
    path : str = Field(alias="host_id")

    @field_validator("path")
    @classmethod
    def set_path(cls, value):
        host_id = value
        return f"/videos/{host_id}"

index = VideoIndex(**video_dict)
index.dict()

{'objectID': 'VO8Bk03Xv90',
 'title': 'Obi-Wan - Obi-Wan vs Darth Vader - (HDR )',
 'path': '/videos/VO8Bk03Xv90'}

In [6]:
statement = select (Playlist)
playlist_db = await session.exec(statement)
playlist = playlist_db.first()
playlist_dict = playlist.__dict__
playlist_dict

2024-07-03 12:11:28,249 INFO sqlalchemy.engine.Engine SELECT playlist.db_id, playlist.owner_id, playlist.host_ids, playlist.updated, playlist.title 
FROM playlist
2024-07-03 12:11:28,269 INFO sqlalchemy.engine.Engine [generated in 0.01998s] ()


{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState at 0x7fd3318a0ad0>,
 'owner_id': UUID('dca2ebb8-33c6-11ef-af11-00163e90c528'),
 'db_id': UUID('5afc9592-346a-11ef-a94b-00163e90c528'),
 'title': 'Hello world',
 'updated': datetime.datetime(2024, 6, 28, 7, 39, 39, 415521, tzinfo=datetime.timezone.utc),
 'host_ids': ['VO8Bk03Xv90', 'q9NuIVwYa9Q']}

In [7]:
class PlaylistIndex(BaseModel):
    objectID : UUID = Field(alias='db_id')
    title : Optional[str]
    path : str = Field(default="/")

    @model_validator(mode='after')
    def set_defaults(self):
        self.objectID = str(self.objectID)
        self.path = f"/playlist/{self.objectID}"
        return self

PlaylistIndex(**playlist_dict).dict()

/home/davidhenao695/Python/Backend/FastAPI/video_membership/venv/lib/python3.11/site-packages/pydantic/main.py:347: UserWarning: Pydantic serializer warnings:
  Expected `uuid` but got `str` - serialized value may not be as expected
  return self.__pydantic_serializer__.to_python(


{'objectID': '5afc9592-346a-11ef-a94b-00163e90c528',
 'title': 'Hello world',
 'path': '/playlist/5afc9592-346a-11ef-a94b-00163e90c528'}

In [8]:
statement = select (Video)
video_db = await session.exec(statement)
video_list = video_db.all()

video_dic = [x.as_dict() for x in video_list]
video_dataset = [VideoIndex(**x).dict() for x in video_dic]
video_dataset

2024-07-03 12:11:30,098 INFO sqlalchemy.engine.Engine SELECT videos.db_id, videos.host_service, videos.host_id, videos.url, videos.title, videos.owner_id 
FROM videos
2024-07-03 12:11:30,103 INFO sqlalchemy.engine.Engine [cached since 2.593s ago] ()


[{'objectID': 'VO8Bk03Xv90',
  'title': 'Obi-Wan - Obi-Wan vs Darth Vader - (HDR )',
  'path': '/videos/VO8Bk03Xv90'},
 {'objectID': 'q9NuIVwYa9Q',
  'title': 'Los Espiritus - Gratitud Full Album',
  'path': '/videos/q9NuIVwYa9Q'},
 {'objectID': '7OlzxfSKjtI',
  'title': 'DIVIDIDOS - Guanuqueando',
  'path': '/videos/7OlzxfSKjtI'},
 {'objectID': 'xxKjh_bLnQQ',
  'title': 'Acoustic France (Official Putumayo Version)',
  'path': '/videos/xxKjh_bLnQQ'},
 {'objectID': 'p_KxCQq7jUI',
  'title': 'Italian Café (Official Putumayo Version)',
  'path': '/videos/p_KxCQq7jUI'}]

In [9]:
statement = select (Playlist)
playlist_db = await session.exec(statement)
playlist_list = playlist_db.all()

playlist_dic = [x.as_dict() for x in playlist_list]
playlist_dataset = [PlaylistIndex(**x).dict() for x in playlist_dic]
playlist_dataset

2024-07-03 12:11:32,293 INFO sqlalchemy.engine.Engine SELECT playlist.db_id, playlist.owner_id, playlist.host_ids, playlist.updated, playlist.title 
FROM playlist
2024-07-03 12:11:32,299 INFO sqlalchemy.engine.Engine [cached since 4.051s ago] ()


[{'objectID': '5afc9592-346a-11ef-a94b-00163e90c528',
  'title': 'Hello world',
  'path': '/playlist/5afc9592-346a-11ef-a94b-00163e90c528'},
 {'objectID': '2e13270e-3531-11ef-9bde-00163e90c528',
  'title': 'Hello 3',
  'path': '/playlist/2e13270e-3531-11ef-9bde-00163e90c528'},
 {'objectID': '254cda3a-348b-11ef-9ced-00163e90c528',
  'title': 'Hello world2',
  'path': '/playlist/254cda3a-348b-11ef-9ced-00163e90c528'}]

In [11]:
dataset = playlist_dataset + video_dataset
dataset

[{'objectID': '5afc9592-346a-11ef-a94b-00163e90c528',
  'title': 'Hello world',
  'path': '/playlist/5afc9592-346a-11ef-a94b-00163e90c528'},
 {'objectID': '2e13270e-3531-11ef-9bde-00163e90c528',
  'title': 'Hello 3',
  'path': '/playlist/2e13270e-3531-11ef-9bde-00163e90c528'},
 {'objectID': '254cda3a-348b-11ef-9ced-00163e90c528',
  'title': 'Hello world2',
  'path': '/playlist/254cda3a-348b-11ef-9ced-00163e90c528'},
 {'objectID': 'VO8Bk03Xv90',
  'title': 'Obi-Wan - Obi-Wan vs Darth Vader - (HDR )',
  'path': '/videos/VO8Bk03Xv90'},
 {'objectID': 'q9NuIVwYa9Q',
  'title': 'Los Espiritus - Gratitud Full Album',
  'path': '/videos/q9NuIVwYa9Q'},
 {'objectID': '7OlzxfSKjtI',
  'title': 'DIVIDIDOS - Guanuqueando',
  'path': '/videos/7OlzxfSKjtI'},
 {'objectID': 'xxKjh_bLnQQ',
  'title': 'Acoustic France (Official Putumayo Version)',
  'path': '/videos/xxKjh_bLnQQ'},
 {'objectID': 'p_KxCQq7jUI',
  'title': 'Italian Café (Official Putumayo Version)',
  'path': '/videos/p_KxCQq7jUI'}]